In [309]:
import math 
import torch
import os
import pandas as pd 
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset
from torchvision.models import resnet18
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
import wget
import numpy as np
from skimage import io
from skimage.transform import resize
import co

In [63]:
path_to_download = os.getcwd()
path_to_download = os.path.join(path_to_download, 'data')
path_to_download = os.path.join(path_to_download, 'photos_4500(3950)_v_6.csv')
df = pd.read_csv(path_to_download)
df

,Picture URL,Amount of likes,Author name,Amount of subscribers,Post tags,Post date,Download URL,Marker
0,https://www.instagram.com/p/CNzI3m9MNxO/,165,romina_stanescu,1 397,"['#morningselfie', '#morningmood', '#selfie']",2021-04-18T07:32:21.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
1,https://www.instagram.com/p/CNzKR83HAvI/,1 217,yeonions,299 936,"['#fromo', '#남자스니커즈', '#여자스니커즈', '#화이트스니커즈', '...",2021-04-18T07:44:42.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
2,https://www.instagram.com/p/CNzCpoGn1_U/,1 441,hyo._.jin23,63 119,"['#네이밍미', '#올리비엣스퀘어드레스']",2021-04-18T06:38:01.000Z,https://scontent-arn2-2.cdninstagram.com/v/t51...,0
3,https://www.instagram.com/p/CNzNd_MJIU6/,367,out___and___outer,10 961,"['#travel', '#модельмосква', '#модельдубай', '...",2021-04-18T08:12:33.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
4,https://www.instagram.com/p/CNzOicxHVOT/,1 479,all_zero._.k,104 242,[],2021-04-18T08:21:54.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
...,...,...,...,...,...,...,...,...
3946,https://www.instagram.com/p/CN2IA8Jnr4C/,828,dislav_v,8 403,"['#in', '#instagram', '#insta', '#lol', '#1', ...",2021-04-19T11:22:37.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
3947,https://www.instagram.com/p/CN2IAvtFWZW/,31,joshuabravo,765,"['#mensfashion', '#fashion', '#style', '#mensw...",2021-04-19T11:22:35.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
3948,https://www.instagram.com/p/CN2IAYisu4A/,38,mala.mi.trenuje,1 411,"['#rozRUSZAMY', '#polishgirl', '#fitgirl', '#f...",2021-04-19T11:22:32.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
3949,https://www.instagram.com/p/CN2IAS-jrh-/,33,tushargedar,360,"['#likes', '#like', '#follow', '#likeforlikes'...",2021-04-19T11:22:32.000Z,https://scontent-arn2-2.cdninstagram.com/v/t51...,0


In [64]:
df['Amount of likes'] = df['Amount of likes'].str.replace(' ', '')
df['Amount of subscribers'] = df['Amount of subscribers'].str.replace(' ', '')

df['Amount of likes'] = df['Amount of likes'].astype(int)
df['Amount of subscribers'] = df['Amount of subscribers'].astype(int)


df

,Picture URL,Amount of likes,Author name,Amount of subscribers,Post tags,Post date,Download URL,Marker
0,https://www.instagram.com/p/CNzI3m9MNxO/,165,romina_stanescu,1397,"['#morningselfie', '#morningmood', '#selfie']",2021-04-18T07:32:21.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
1,https://www.instagram.com/p/CNzKR83HAvI/,1217,yeonions,299936,"['#fromo', '#남자스니커즈', '#여자스니커즈', '#화이트스니커즈', '...",2021-04-18T07:44:42.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
2,https://www.instagram.com/p/CNzCpoGn1_U/,1441,hyo._.jin23,63119,"['#네이밍미', '#올리비엣스퀘어드레스']",2021-04-18T06:38:01.000Z,https://scontent-arn2-2.cdninstagram.com/v/t51...,0
3,https://www.instagram.com/p/CNzNd_MJIU6/,367,out___and___outer,10961,"['#travel', '#модельмосква', '#модельдубай', '...",2021-04-18T08:12:33.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
4,https://www.instagram.com/p/CNzOicxHVOT/,1479,all_zero._.k,104242,[],2021-04-18T08:21:54.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
...,...,...,...,...,...,...,...,...
3946,https://www.instagram.com/p/CN2IA8Jnr4C/,828,dislav_v,8403,"['#in', '#instagram', '#insta', '#lol', '#1', ...",2021-04-19T11:22:37.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
3947,https://www.instagram.com/p/CN2IAvtFWZW/,31,joshuabravo,765,"['#mensfashion', '#fashion', '#style', '#mensw...",2021-04-19T11:22:35.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
3948,https://www.instagram.com/p/CN2IAYisu4A/,38,mala.mi.trenuje,1411,"['#rozRUSZAMY', '#polishgirl', '#fitgirl', '#f...",2021-04-19T11:22:32.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0
3949,https://www.instagram.com/p/CN2IAS-jrh-/,33,tushargedar,360,"['#likes', '#like', '#follow', '#likeforlikes'...",2021-04-19T11:22:32.000Z,https://scontent-arn2-2.cdninstagram.com/v/t51...,0


In [65]:
df['Ratio'] = np.nan
df

,Picture URL,Amount of likes,Author name,Amount of subscribers,Post tags,Post date,Download URL,Marker,Ratio
0,https://www.instagram.com/p/CNzI3m9MNxO/,165,romina_stanescu,1397,"['#morningselfie', '#morningmood', '#selfie']",2021-04-18T07:32:21.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,NaN
1,https://www.instagram.com/p/CNzKR83HAvI/,1217,yeonions,299936,"['#fromo', '#남자스니커즈', '#여자스니커즈', '#화이트스니커즈', '...",2021-04-18T07:44:42.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,NaN
2,https://www.instagram.com/p/CNzCpoGn1_U/,1441,hyo._.jin23,63119,"['#네이밍미', '#올리비엣스퀘어드레스']",2021-04-18T06:38:01.000Z,https://scontent-arn2-2.cdninstagram.com/v/t51...,0,NaN
3,https://www.instagram.com/p/CNzNd_MJIU6/,367,out___and___outer,10961,"['#travel', '#модельмосква', '#модельдубай', '...",2021-04-18T08:12:33.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,NaN
4,https://www.instagram.com/p/CNzOicxHVOT/,1479,all_zero._.k,104242,[],2021-04-18T08:21:54.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,NaN
...,...,...,...,...,...,...,...,...,...
3946,https://www.instagram.com/p/CN2IA8Jnr4C/,828,dislav_v,8403,"['#in', '#instagram', '#insta', '#lol', '#1', ...",2021-04-19T11:22:37.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,NaN
3947,https://www.instagram.com/p/CN2IAvtFWZW/,31,joshuabravo,765,"['#mensfashion', '#fashion', '#style', '#mensw...",2021-04-19T11:22:35.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,NaN
3948,https://www.instagram.com/p/CN2IAYisu4A/,38,mala.mi.trenuje,1411,"['#rozRUSZAMY', '#polishgirl', '#fitgirl', '#f...",2021-04-19T11:22:32.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,NaN
3949,https://www.instagram.com/p/CN2IAS-jrh-/,33,tushargedar,360,"['#likes', '#like', '#follow', '#likeforlikes'...",2021-04-19T11:22:32.000Z,https://scontent-arn2-2.cdninstagram.com/v/t51...,0,NaN


In [66]:
number = 0

for i in range (0, 3951):
    if arr[i] >= 0.95:
        number += 1

        
print(number)

92


In [67]:
for i in range (0, len(df)):
    ratio = df['Amount of likes'][i] / df['Amount of subscribers'][i]
    if ratio <= 0.95:
        df['Ratio'][i] = ratio
        
df

c:\users\gafed\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\users\gafed\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in long_scalars
  


,Picture URL,Amount of likes,Author name,Amount of subscribers,Post tags,Post date,Download URL,Marker,Ratio
0,https://www.instagram.com/p/CNzI3m9MNxO/,165,romina_stanescu,1397,"['#morningselfie', '#morningmood', '#selfie']",2021-04-18T07:32:21.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.118110
1,https://www.instagram.com/p/CNzKR83HAvI/,1217,yeonions,299936,"['#fromo', '#남자스니커즈', '#여자스니커즈', '#화이트스니커즈', '...",2021-04-18T07:44:42.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.004058
2,https://www.instagram.com/p/CNzCpoGn1_U/,1441,hyo._.jin23,63119,"['#네이밍미', '#올리비엣스퀘어드레스']",2021-04-18T06:38:01.000Z,https://scontent-arn2-2.cdninstagram.com/v/t51...,0,0.022830
3,https://www.instagram.com/p/CNzNd_MJIU6/,367,out___and___outer,10961,"['#travel', '#модельмосква', '#модельдубай', '...",2021-04-18T08:12:33.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.033482
4,https://www.instagram.com/p/CNzOicxHVOT/,1479,all_zero._.k,104242,[],2021-04-18T08:21:54.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.014188
...,...,...,...,...,...,...,...,...,...
3946,https://www.instagram.com/p/CN2IA8Jnr4C/,828,dislav_v,8403,"['#in', '#instagram', '#insta', '#lol', '#1', ...",2021-04-19T11:22:37.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.098536
3947,https://www.instagram.com/p/CN2IAvtFWZW/,31,joshuabravo,765,"['#mensfashion', '#fashion', '#style', '#mensw...",2021-04-19T11:22:35.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.040523
3948,https://www.instagram.com/p/CN2IAYisu4A/,38,mala.mi.trenuje,1411,"['#rozRUSZAMY', '#polishgirl', '#fitgirl', '#f...",2021-04-19T11:22:32.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.026931
3949,https://www.instagram.com/p/CN2IAS-jrh-/,33,tushargedar,360,"['#likes', '#like', '#follow', '#likeforlikes'...",2021-04-19T11:22:32.000Z,https://scontent-arn2-2.cdninstagram.com/v/t51...,0,0.091667


In [68]:
df[df['Ratio'].isin([np.nan])]

,Picture URL,Amount of likes,Author name,Amount of subscribers,Post tags,Post date,Download URL,Marker,Ratio
84,https://www.instagram.com/p/CM968q5BZWI/,106,lisa_ay_me,38,"['#sunday', '#viral', '#instafashion', '#fashi...",2021-03-28T15:30:58.000Z,https://scontent-arn2-2.cdninstagram.com/v/t51...,0,NaN
85,https://www.instagram.com/p/CJ0u5UyHiKc/,1546,hi_my_name_is_yura_,1555,"['#selfie', '#selca', '#daily', '#dailylook', ...",2021-01-09T12:18:18.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,NaN
175,https://www.instagram.com/p/CNw66Z6hbzK/,177,kiol_wx,181,[],2021-04-17T10:51:55.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,NaN
256,https://www.instagram.com/p/CNzSI78gLmy/,40,rj21group_0017,39,"['#happy', '#style', '#life', '#photo', '#natu...",2021-04-18T08:53:22.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,NaN
442,https://www.instagram.com/p/CNzR5c1J-_d/,15,ullasp_u12,13,"['#cuteboy', '#cute', '#love', '#boy', '#hands...",2021-04-18T08:51:15.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,NaN
...,...,...,...,...,...,...,...,...,...
3712,https://www.instagram.com/p/CN2IRtnBrOn/,4,pb0____v.i.p,0,"['#likes', '#like', '#follow', '#likeforlikes'...",2021-04-19T11:24:54.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,NaN
3751,https://www.instagram.com/p/CKOPkEin4f0/,42,natural_hd3,43,"['#love', '#lovequotes', '#forest', '#food', '...",2021-01-19T10:04:47.000Z,https://scontent-arn2-2.cdninstagram.com/v/t51...,0,NaN
3762,https://www.instagram.com/p/CN2IOM2FUFk/,3,_fashion_grapher,0,"['#fashion', '#love', '#style', '#instagood', ...",2021-04-19T11:24:26.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,NaN
3849,https://www.instagram.com/p/CN2IH-BrO_c/,4,best_shariye,4,"['#love', '#instagood', '#photooftheday', '#fa...",2021-04-19T11:23:35.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,NaN


In [69]:
df = df[df['Ratio'].notna()]
df.reset_index(inplace=True, drop=True) 
df

,Picture URL,Amount of likes,Author name,Amount of subscribers,Post tags,Post date,Download URL,Marker,Ratio
0,https://www.instagram.com/p/CNzI3m9MNxO/,165,romina_stanescu,1397,"['#morningselfie', '#morningmood', '#selfie']",2021-04-18T07:32:21.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.118110
1,https://www.instagram.com/p/CNzKR83HAvI/,1217,yeonions,299936,"['#fromo', '#남자스니커즈', '#여자스니커즈', '#화이트스니커즈', '...",2021-04-18T07:44:42.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.004058
2,https://www.instagram.com/p/CNzCpoGn1_U/,1441,hyo._.jin23,63119,"['#네이밍미', '#올리비엣스퀘어드레스']",2021-04-18T06:38:01.000Z,https://scontent-arn2-2.cdninstagram.com/v/t51...,0,0.022830
3,https://www.instagram.com/p/CNzNd_MJIU6/,367,out___and___outer,10961,"['#travel', '#модельмосква', '#модельдубай', '...",2021-04-18T08:12:33.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.033482
4,https://www.instagram.com/p/CNzOicxHVOT/,1479,all_zero._.k,104242,[],2021-04-18T08:21:54.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.014188
...,...,...,...,...,...,...,...,...,...
3854,https://www.instagram.com/p/CN2IA8Jnr4C/,828,dislav_v,8403,"['#in', '#instagram', '#insta', '#lol', '#1', ...",2021-04-19T11:22:37.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.098536
3855,https://www.instagram.com/p/CN2IAvtFWZW/,31,joshuabravo,765,"['#mensfashion', '#fashion', '#style', '#mensw...",2021-04-19T11:22:35.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.040523
3856,https://www.instagram.com/p/CN2IAYisu4A/,38,mala.mi.trenuje,1411,"['#rozRUSZAMY', '#polishgirl', '#fitgirl', '#f...",2021-04-19T11:22:32.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.026931
3857,https://www.instagram.com/p/CN2IAS-jrh-/,33,tushargedar,360,"['#likes', '#like', '#follow', '#likeforlikes'...",2021-04-19T11:22:32.000Z,https://scontent-arn2-2.cdninstagram.com/v/t51...,0,0.091667


c:\users\gafed\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Picture URL,Amount of likes,Author name,Amount of subscribers,Post tags,Post date,Download URL,Marker,Ratio,Class
0,https://www.instagram.com/p/CNzI3m9MNxO/,165,romina_stanescu,1397,"['#morningselfie', '#morningmood', '#selfie']",2021-04-18T07:32:21.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.118110,NaN
1,https://www.instagram.com/p/CNzKR83HAvI/,1217,yeonions,299936,"['#fromo', '#남자스니커즈', '#여자스니커즈', '#화이트스니커즈', '...",2021-04-18T07:44:42.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.004058,NaN
2,https://www.instagram.com/p/CNzCpoGn1_U/,1441,hyo._.jin23,63119,"['#네이밍미', '#올리비엣스퀘어드레스']",2021-04-18T06:38:01.000Z,https://scontent-arn2-2.cdninstagram.com/v/t51...,0,0.022830,NaN
3,https://www.instagram.com/p/CNzNd_MJIU6/,367,out___and___outer,10961,"['#travel', '#модельмосква', '#модельдубай', '...",2021-04-18T08:12:33.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.033482,NaN
4,https://www.instagram.com/p/CNzOicxHVOT/,1479,all_zero._.k,104242,[],2021-04-18T08:21:54.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.014188,NaN
...,...,...,...,...,...,...,...,...,...,...
3854,https://www.instagram.com/p/CN2IA8Jnr4C/,828,dislav_v,8403,"['#in', '#instagram', '#insta', '#lol', '#1', ...",2021-04-19T11:22:37.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.098536,NaN
3855,https://www.instagram.com/p/CN2IAvtFWZW/,31,joshuabravo,765,"['#mensfashion', '#fashion', '#style', '#mensw...",2021-04-19T11:22:35.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.040523,NaN
3856,https://www.instagram.com/p/CN2IAYisu4A/,38,mala.mi.trenuje,1411,"['#rozRUSZAMY', '#polishgirl', '#fitgirl', '#f...",2021-04-19T11:22:32.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.026931,NaN
3857,https://www.instagram.com/p/CN2IAS-jrh-/,33,tushargedar,360,"['#likes', '#like', '#follow', '#likeforlikes'...",2021-04-19T11:22:32.000Z,https://scontent-arn2-2.cdninstagram.com/v/t51...,0,0.091667,NaN


In [75]:
for i in range (0, len(df)):
    num = df['Ratio'][i] * 10
    df['Class'][i] = int(num + (0.5 if num > 0 else -0.5))
    
df

c:\users\gafed\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\gafed\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:3331: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)


,Picture URL,Amount of likes,Author name,Amount of subscribers,Post tags,Post date,Download URL,Marker,Ratio,Class
0,https://www.instagram.com/p/CNzI3m9MNxO/,165,romina_stanescu,1397,"['#morningselfie', '#morningmood', '#selfie']",2021-04-18T07:32:21.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.118110,1.0
1,https://www.instagram.com/p/CNzKR83HAvI/,1217,yeonions,299936,"['#fromo', '#남자스니커즈', '#여자스니커즈', '#화이트스니커즈', '...",2021-04-18T07:44:42.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.004058,0.0
2,https://www.instagram.com/p/CNzCpoGn1_U/,1441,hyo._.jin23,63119,"['#네이밍미', '#올리비엣스퀘어드레스']",2021-04-18T06:38:01.000Z,https://scontent-arn2-2.cdninstagram.com/v/t51...,0,0.022830,0.0
3,https://www.instagram.com/p/CNzNd_MJIU6/,367,out___and___outer,10961,"['#travel', '#модельмосква', '#модельдубай', '...",2021-04-18T08:12:33.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.033482,0.0
4,https://www.instagram.com/p/CNzOicxHVOT/,1479,all_zero._.k,104242,[],2021-04-18T08:21:54.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.014188,0.0
...,...,...,...,...,...,...,...,...,...,...
3854,https://www.instagram.com/p/CN2IA8Jnr4C/,828,dislav_v,8403,"['#in', '#instagram', '#insta', '#lol', '#1', ...",2021-04-19T11:22:37.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.098536,1.0
3855,https://www.instagram.com/p/CN2IAvtFWZW/,31,joshuabravo,765,"['#mensfashion', '#fashion', '#style', '#mensw...",2021-04-19T11:22:35.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.040523,0.0
3856,https://www.instagram.com/p/CN2IAYisu4A/,38,mala.mi.trenuje,1411,"['#rozRUSZAMY', '#polishgirl', '#fitgirl', '#f...",2021-04-19T11:22:32.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.026931,0.0
3857,https://www.instagram.com/p/CN2IAS-jrh-/,33,tushargedar,360,"['#likes', '#like', '#follow', '#likeforlikes'...",2021-04-19T11:22:32.000Z,https://scontent-arn2-2.cdninstagram.com/v/t51...,0,0.091667,1.0


In [84]:
for i in range (0, 10):
    print(str(i) + " " + str(len(df[df['Class'].isin([i])])))

0 1825
1 1223
2 411
3 204
4 73
5 53
6 27
7 14
8 16
9 13


In [85]:
def save_df():
    path_to_saxe_df = os.getcwd()
    path_to_saxe_df = os.path.join(path_to_saxe_df, "selfies.csv")
    df.to_csv(r'{0}'.format(path_to_saxe_df), index = False)

In [86]:
save_df()

In [89]:
df[50:100]

,Picture URL,Amount of likes,Author name,Amount of subscribers,Post tags,Post date,Download URL,Marker,Ratio,Class
50,https://www.instagram.com/p/CNzMANMrT4E/,161,0.04k_,415,[],2021-04-18T07:59:45.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.387952,4.0
51,https://www.instagram.com/p/CNy7EgSnSMo/,294,ziminnx,1941,[],2021-04-18T05:31:47.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.151468,2.0
52,https://www.instagram.com/p/CNy0_7knqSD/,166,cherry2ma,1779,[],2021-04-18T04:38:44.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.093311,1.0
53,https://www.instagram.com/p/CNy0wwYsIdT/,149,98__hyun2,339,[],2021-04-18T04:36:40.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.439528,4.0
54,https://www.instagram.com/p/CNyLUyJLGjl/,175,sedaggll,2391,"['#igers', '#instacool', '#instamood', '#me', ...",2021-04-17T22:34:35.000Z,https://scontent-arn2-2.cdninstagram.com/v/t51...,0,0.073191,1.0
55,https://www.instagram.com/p/CNyLUyJLGjl/,175,sedaggll,2391,"['#igers', '#instacool', '#instamood', '#me', ...",2021-04-17T22:34:35.000Z,https://scontent-arn2-2.cdninstagram.com/v/t51...,0,0.073191,1.0
56,https://www.instagram.com/p/CJzjOyhnJVw/,1473,hi_my_name_is_yura_,1555,"['#selfie', '#selca', '#daily', '#dailylook', ...",2021-01-09T01:17:08.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.947267,9.0
57,https://www.instagram.com/p/CNzTTEjnG-t/,15,annegienff,1680,"['#glassesgirl', '#pullandbear', '#blondehair'...",2021-04-18T09:03:29.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.008929,0.0
58,https://www.instagram.com/p/CNzTS77Lw2B/,7,quatresnuku,173,"['#sewing', '#imadeit', '#selfie', '#homemade'...",2021-04-18T09:03:28.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.040462,0.0
59,https://www.instagram.com/p/CNzTS5Epl8t/,9,basilejoelle,266,"['#straiching', '#girl', '#selfie', '#rush', '...",2021-04-18T09:03:28.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.033835,0.0


In [116]:
df[df['Class'].isin([0])]

,Picture URL,Amount of likes,Author name,Amount of subscribers,Post tags,Post date,Download URL,Marker,Ratio,Class
1,https://www.instagram.com/p/CNzKR83HAvI/,1217,yeonions,299936,"['#fromo', '#남자스니커즈', '#여자스니커즈', '#화이트스니커즈', '...",2021-04-18T07:44:42.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.004058,0.0
2,https://www.instagram.com/p/CNzCpoGn1_U/,1441,hyo._.jin23,63119,"['#네이밍미', '#올리비엣스퀘어드레스']",2021-04-18T06:38:01.000Z,https://scontent-arn2-2.cdninstagram.com/v/t51...,0,0.022830,0.0
3,https://www.instagram.com/p/CNzNd_MJIU6/,367,out___and___outer,10961,"['#travel', '#модельмосква', '#модельдубай', '...",2021-04-18T08:12:33.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.033482,0.0
4,https://www.instagram.com/p/CNzOicxHVOT/,1479,all_zero._.k,104242,[],2021-04-18T08:21:54.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.014188,0.0
5,https://www.instagram.com/p/CNzIA0gH2lu/,282,wi.__.iw,10113,[],2021-04-18T07:24:53.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.027885,0.0
...,...,...,...,...,...,...,...,...,...,...
3852,https://www.instagram.com/p/CN2IA_UnX9m/,17,diyrow,2188,"['#diy', '#diyvideos', '#tutorial', '#diyforli...",2021-04-19T11:22:37.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.007770,0.0
3853,https://www.instagram.com/p/CN2IA_AnS8f/,13,its_sourabh_777,1433,"['#sourabhsarkar😎', '#royal', '#love', '#insta...",2021-04-19T11:22:37.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.009072,0.0
3855,https://www.instagram.com/p/CN2IAvtFWZW/,31,joshuabravo,765,"['#mensfashion', '#fashion', '#style', '#mensw...",2021-04-19T11:22:35.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.040523,0.0
3856,https://www.instagram.com/p/CN2IAYisu4A/,38,mala.mi.trenuje,1411,"['#rozRUSZAMY', '#polishgirl', '#fitgirl', '#f...",2021-04-19T11:22:32.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.026931,0.0


In [110]:
df.iloc[1]

Picture URL                       https://www.instagram.com/p/CNzKR83HAvI/
Amount of likes                                                       1217
Author name                                                       yeonions
Amount of subscribers                                               299936
Post tags                ['#fromo', '#남자스니커즈', '#여자스니커즈', '#화이트스니커즈', '...
Post date                                         2021-04-18T07:44:42.000Z
Download URL             https://scontent-arn2-1.cdninstagram.com/v/t51...
Marker                                                                   0
Ratio                                                           0.00405753
Class                                                                    0
Name: 1, dtype: object

In [269]:
num_0 = 0
num_1 = 0
some_df = pd.DataFrame(columns = ['Picture URL' , 'Amount of likes', 'Author name', 'Amount of subscribers', 
                        'Post tags', 'Post date', 'Download URL', 'Marker', 'Ratio', 'Class', 'ID'])

for i in range(0, len(df)):
    if df['Class'][i] == 0.0 and num_0 < 10:
        (some_df.loc)[num_0] = (df.loc)[i]
        (some_df['ID'])[num_0] = i
        num_0 += 1

for i in range(0, len(df)):
    if df['Class'][i] == 2.0 and num_1 < 10:
        (some_df.loc)[num_0 + num_1] = (df.loc)[i]
        (some_df['ID'])[num_0 + num_1] = i
        num_1 += 1


some_df    

c:\users\gafed\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
c:\users\gafed\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,Picture URL,Amount of likes,Author name,Amount of subscribers,Post tags,Post date,Download URL,Marker,Ratio,Class,ID
0,https://www.instagram.com/p/CNzKR83HAvI/,1217,yeonions,299936,"['#fromo', '#남자스니커즈', '#여자스니커즈', '#화이트스니커즈', '...",2021-04-18T07:44:42.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.004058,0.0,1.0
1,https://www.instagram.com/p/CNzCpoGn1_U/,1441,hyo._.jin23,63119,"['#네이밍미', '#올리비엣스퀘어드레스']",2021-04-18T06:38:01.000Z,https://scontent-arn2-2.cdninstagram.com/v/t51...,0,0.022830,0.0,2.0
2,https://www.instagram.com/p/CNzNd_MJIU6/,367,out___and___outer,10961,"['#travel', '#модельмосква', '#модельдубай', '...",2021-04-18T08:12:33.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.033482,0.0,3.0
3,https://www.instagram.com/p/CNzOicxHVOT/,1479,all_zero._.k,104242,[],2021-04-18T08:21:54.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.014188,0.0,4.0
4,https://www.instagram.com/p/CNzIA0gH2lu/,282,wi.__.iw,10113,[],2021-04-18T07:24:53.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.027885,0.0,5.0
5,https://www.instagram.com/p/CNzSZX_Bhm-/,4709,lizkaeber,958143,['#selfie'],2021-04-18T08:55:37.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.004915,0.0,6.0
6,https://www.instagram.com/p/CNzK_5JB4PY/,208,kengjjang1,4282,"['#울릉도', '#독도', '#성인봉', '#독도에갇힌행복한섬처녀다', '#이야호...",2021-04-18T07:50:58.000Z,https://scontent-arn2-2.cdninstagram.com/v/t51...,0,0.048575,0.0,7.0
7,https://www.instagram.com/p/CNzOatsHUML/,442,ju_luv__,58164,[],2021-04-18T08:20:50.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.007599,0.0,8.0
8,https://www.instagram.com/p/CNzPuehnUkq/,616,mysteriegla,63320,[],2021-04-18T08:32:17.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.009728,0.0,9.0
9,https://www.instagram.com/p/CNzBScirbSt/,125,it.is.makarenko,9208,"['#selfie', '#moscow', '#weekend', '#spring', ...",2021-04-18T06:26:07.000Z,https://scontent-arn2-1.cdninstagram.com/v/t51...,0,0.013575,0.0,10.0


In [270]:
path = os.getcwd()
path = os.path.join(path, 'photoset')

def save_picture(i):
    try:
        save_as = os.path.join(path, str((some_df['ID'].loc)[i]) + '.jpg')
        wget.download(some_df['Download URL'][i], save_as)
+        some_df['Marker'][i] = 1  

In [196]:
a = int((some_df['ID'].loc)[0])
a

1

In [320]:
def save_df_for_resnet():
    path_to_saxe_df = os.getcwd()
    path_to_saxe_df = os.path.join(path_to_saxe_df, "df_for_resnet.csv")
    df_for_resnet.to_csv(r'{0}'.format(path_to_saxe_df), index = False)